In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [5]:
filepath_dict= {'yelp':   './data/yelp_labelled.txt',
                 'amazon': './data/amazon_cells_labelled.txt',
                 'imdb':   './data/imdb_labelled.txt'}
df_list = []

In [6]:
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source 
    df_list.append(df)
df = pd.concat(df_list)
df.head(5)

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


### X_test, y_test splitting related models's accuracy

In [13]:
df_yelp = df[df['source'] == 'yelp']
X = df_yelp['sentence'].values
y = df_yelp['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)
len(X_train)



 just to get some carbs in your stomach.',
       'We watched our waiter pay a lot more attention to other tables and ignore us.',
       'The chips and sals a here is amazing!!!!!!!!!!!!!!!!!!!',
       'Everything on the menu is terrific and we were also thrilled that they made amazing accommodations for our vegetarian daughter.',
       'The block was amazing.',
       'The food is delicious and just spicy enough, so be sure to ask for spicier if you prefer it that way.',
       "I won't be back.",
       'The ripped banana was not only ripped, but petrified and tasteless.',
       'The food is very good for your typical bar food.',
       'We waited for forty five minutes in vain.',
       'But then they came back cold.',
       'We got the food and apparently they have never heard of salt and the batter on the fish was chewy.',
       'They have great dinners.', 'The service was a bit lacking.',
       'We enjoy their pizza and brunch.', 'I was seated immediately.',
       'Servic